In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import datetime as dt

In [2]:
import plotly
import plotly.express as px

import plotly.graph_objs as go
from plotly.subplots import make_subplots


plotly.offline.init_notebook_mode(connected=True)

In [3]:
import cvxpy as cp

In [61]:
Prices = pd.read_pickle('ETF_Prices.pkl')
Returns = Prices.pct_change()            # Converts prices to returns
Returns.mean()

EEM    0.007220
IVV    0.004115
IWM    0.005639
EFA    0.003924
dtype: float64

In [70]:
r = np.array(12*Returns.mean())
sigma = np.array(np.sqrt(12) * Returns.cov())

In [117]:
# Optimization setup
n = len(Returns.columns)     # number of assets

w = cp.Variable(n)           # Weights vector

ret = (r.T)*w                         # Portfolio return term in objective
variance = cp.quad_form(w,sigma)      # Risk term
RiskAv  = cp.Parameter(nonneg=True)

U = ret - RiskAv*variance
objective = cp.Maximize(U)

constraints = [ w >= 0, w <= 0.5,
                cp.sum(w) == 1]

prob = cp.Problem(objective,constraints)

# Looping to construct a return vs risk frontier

nPorts = 100
Res = np.zeros((nPorts,n+4))
for i in range(nPorts):
    RiskAv.value = i *0.2                
    _ = prob.solve()
    
    Res[i] = np.append(w.value,(ret.value,np.sqrt(variance.value),RiskAv.value,U.value))

# Collect results in useful form
ResDF = pd.DataFrame(Res,columns=list(Returns.columns)+['Return','Risk','RiskAv','Utility'])

RiskFree = 1.8/100              # As of August 9, 2019
ResDF['SharpeRatio'] = (ResDF['Return']-RiskFree)/ResDF['Risk']

In [132]:
ResDF.tail()

,EEM,IVV,IWM,EFA,Return,Risk,RiskAv,Utility,SharpeRatio
95,0.150670,0.5,0.183443,0.165886,0.057968,0.073087,19.0,-0.043524,0.546856
96,0.146767,0.5,0.181957,0.171276,0.057783,0.073021,19.2,-0.044592,0.544819
97,0.142944,0.5,0.180501,0.176555,0.057602,0.072956,19.4,-0.045657,0.542815
98,0.139199,0.5,0.179075,0.181726,0.057424,0.072894,19.6,-0.046721,0.540845
99,0.135529,0.5,0.177678,0.186793,0.057250,0.072833,19.8,-0.047783,0.538907


In [131]:
#ResDF.loc[ResDF['SharpeRatio']==ResDF['SharpeRatio'].max(),:]

ResDF.query('SharpeRatio == SharpeRatio.max()')

,EEM,IVV,IWM,EFA,Return,Risk,RiskAv,Utility,SharpeRatio
12,0.500002,0.000007,0.499997,-0.000006,0.077152,0.087891,2.4,0.058613,0.673019


In [112]:
px.line(ResDF,x='Risk',y='Return').update_layout(xaxis=dict(tickformat='.1%'),
                                                 yaxis=dict(tickformat='.1%'))